# 5-3-H Спички детям не игрушка!

In [ ]:
from collections import Counter

with open('test/input-5-3-H.txt', mode='r') as f:

    n = int(f.readline())

    A, B = [], []
    for fig in [A, B]:
        for _ in range(n):
            m = list(map(int, f.readline().split()))
            start = (m[0], m[1])
            end = (m[2], m[3])
            # Унифицируем направление спички (спичка (a, b) = спичка (b, a))
            if start[1] > end[1] or (start[1] == end[1] and end[0] < start[0]):
                start, end = end, start
            vector = (end[0] - start[0], end[1] - start[1])
            #fig.append((start, vector))
            fig.append((start, end))

res_counter = Counter()

for m_a in A:
    for m_b in B:
        offset = (
            (m_b[0][0] - m_a[0][0], m_b[0][1] - m_a[0][1]), 
            (m_b[1][0] - m_a[1][0], m_b[1][1] - m_a[1][1])
        )
        if offset[0] == offset[1]:
            res_counter[offset[0]] += 1

same = res_counter.most_common(1)[0][1] if len(res_counter) else 0
print(n - same)